In [66]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

np.random.seed(42)
tf.random.set_seed(42)

In [67]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [68]:
def class_labels_reassign(age):
    if 1 <= age <= 2:
        return 0
    elif 3 <= age <= 9:
        return 1
    elif 10 <= age <= 20:
        return 2
    elif 21 <= age <= 27:
        return 3
    elif 28 <= age <= 45:
        return 4
    elif 46 <= age <= 65:
        return 5
    else:
        return 6

In [69]:
dataframe = pd.read_csv('DATA/face_detection_label.csv')
dataframe.head()

,image,age,gender,races
0,DATA\UTKFace\Face_Detection\100_0_0_2017011221...,100,0,0
1,DATA\UTKFace\Face_Detection\100_0_0_2017011221...,100,0,0
2,DATA\UTKFace\Face_Detection\100_1_0_2017011018...,100,1,0
3,DATA\UTKFace\Face_Detection\100_1_0_2017011221...,100,1,0
4,DATA\UTKFace\Face_Detection\100_1_0_2017011221...,100,1,0


In [70]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19098 entries, 0 to 19097
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   19098 non-null  object
 1   age     19098 non-null  int64 
 2   gender  19098 non-null  int64 
 3   races   19098 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 596.9+ KB


In [71]:
train_df, test_df = train_test_split(dataframe, test_size=0.2, random_state=42, shuffle=True)
print("Train:", train_df.shape)
print("Test:", test_df.shape)

Train: (15278, 4)
Test: (3820, 4)


In [72]:
train_df['target'] = train_df['age'].map(class_labels_reassign)
test_df['target'] = test_df['age'].map(class_labels_reassign)

In [73]:
print("Train:", train_df.shape)
print("Test:", test_df.shape)

Train: (15278, 5)
Test: (3820, 5)


## Organize Datas

In [74]:
train_filenames_list = list(train_df['image'])
train_labels_list = list(train_df['target'])

test_filenames_list = list(test_df['image'])
test_labels_list = list(test_df['target'])

In [86]:
train_filenames_tensor = tf.constant(train_filenames_list)
train_labels_tensor = tf.constant(train_labels_list)

test_filenames_tensor = tf.constant(test_filenames_list)
test_labels_tensor = tf.constant(test_labels_list)

## Image Greyscale Function


In [76]:
num_classes = 7

IMG_HEIGHT = 200
IMG_WIDTH = 200

def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)
    image_resized = tf.image.resize(image_decoded, [IMG_HEIGHT, IMG_WIDTH])  # Resize ở đây
    label = tf.one_hot(label, num_classes)
    return image_resized, label

In [77]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames_tensor, train_labels_tensor))
train_dataset = train_dataset.map(_parse_function)
train_dataset = train_dataset.batch(512)

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames_tensor, test_labels_tensor))
test_dataset = test_dataset.map(_parse_function)
test_dataset = test_dataset.batch(512)

## CNN Model

In [78]:

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, activation='relu',
                 input_shape=(200, 200, 1)))  # 3rd dim = 1 for grayscale images.
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())

model.add(Dense(132, activation='relu'))

model.add(Dense(7, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 198, 198, 32)      320       
                                                                 
 average_pooling2d_20 (Avera  (None, 99, 99, 32)       0         
 gePooling2D)                                                    
                                                                 
 conv2d_21 (Conv2D)          (None, 97, 97, 64)        18496     
                                                                 
 average_pooling2d_21 (Avera  (None, 48, 48, 64)       0         
 gePooling2D)                                                    
                                                                 
 conv2d_22 (Conv2D)          (None, 46, 46, 128)       73856     
                                                                 
 average_pooling2d_22 (Avera  (None, 23, 23, 128)     

In [79]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
tensorboard = TensorBoard(log_dir="E:/Pycharm/Age-Detection-Using-Neural-Network/logs/cnn_logs")

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    filepath="E:/Pycharm/Age-Detection-Using-Neural-Network/logs/age_model_checkpoint.h5",
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)


In [84]:
model_cnn_history = model.fit(train_dataset,
                                  batch_size=512,
                                  validation_data=test_dataset,
                                  epochs=60,
                                  callbacks=[tensorboard, checkpoint],
                                  shuffle=False
                                  )

Epoch 1/60


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_5/conv2d_20/Relu' defined at (most recent call last):
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\buian\anaconda3\envs\tf-gpu\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\buian\anaconda3